In [25]:
#Data loading and preprocessing

from os import listdir
from os.path import isdir
from PIL import Image
from matplotlib import pyplot
from numpy import savez_compressed
from numpy import asarray
from mtcnn.mtcnn import MTCNN

def extract_face(filename, required_size=(160, 160)):
    # load image from file
    image = Image.open(filename)
    image = image.convert('RGB')
    pixels = asarray(image)
    detector = MTCNN()
    results = detector.detect_faces(pixels)
    #Finding dimensions for the box
    x1, y1, width, height = results[0]['box']
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    # extracting face
    face = pixels[y1:y2, x1:x2]
    # resizing pixels to match with embedding model input size
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = asarray(image)
    print(filename + ' extracted')
    return face_array

def load_faces(directory, train_test=0):
    faces = list()
    if train_test==0:
        files = listdir(directory)[:138]
    else:
        files = listdir(directory)[138:]
    for filename in files:
        path = directory + filename
        face = extract_face(path)
        faces.append(face)
    return faces

def load_dataset(directory, train_test=0):
    faces = load_faces(directory, train_test)
    if train_test==0:
        files = listdir(directory)[:138]
    else:
        files = listdir(directory)[138:]
    labels = [filename[filename.find('.')+1:filename.find('.')+3] for filename in files]
    return asarray(faces), asarray(labels)

#Train Test ratio is 65/35 which corresponds to 138 train images and rest test images
# load train dataset
trainX, trainy = load_dataset('Expression_Data/', 0)
print(trainX.shape, trainy.shape)
# load test dataset
testX, testy = load_dataset('Expression_Data/', 1)
save arrays to one file in compressed format
savez_compressed('expression-dataset.npz', trainX, trainy, testX, testy)

/home/ankitsingh/Desktop/datasets/Expression_Data/TM.AN2.191.tiff extracted
/home/ankitsingh/Desktop/datasets/Expression_Data/KA.FE2.46.tiff extracted
/home/ankitsingh/Desktop/datasets/Expression_Data/UY.HA1.137.tiff extracted
/home/ankitsingh/Desktop/datasets/Expression_Data/KM.HA2.5.tiff extracted
/home/ankitsingh/Desktop/datasets/Expression_Data/TM.SA3.186.tiff extracted
/home/ankitsingh/Desktop/datasets/Expression_Data/NM.AN1.104.tiff extracted
/home/ankitsingh/Desktop/datasets/Expression_Data/KL.FE2.175.tiff extracted
/home/ankitsingh/Desktop/datasets/Expression_Data/NM.AN2.105.tiff extracted
/home/ankitsingh/Desktop/datasets/Expression_Data/KM.NE3.3.tiff extracted
/home/ankitsingh/Desktop/datasets/Expression_Data/MK.SA3.121.tiff extracted
/home/ankitsingh/Desktop/datasets/Expression_Data/KM.SU1.14.tiff extracted
/home/ankitsingh/Desktop/datasets/Expression_Data/KL.HA1.158.tiff extracted
/home/ankitsingh/Desktop/datasets/Expression_Data/UY.AN2.147.tiff extracted
/home/ankitsingh/D

/home/ankitsingh/Desktop/datasets/Expression_Data/TM.SA1.184.tiff extracted
/home/ankitsingh/Desktop/datasets/Expression_Data/KR.NE1.71.tiff extracted
/home/ankitsingh/Desktop/datasets/Expression_Data/MK.DI3.130.tiff extracted
/home/ankitsingh/Desktop/datasets/Expression_Data/MK.SA1.119.tiff extracted
/home/ankitsingh/Desktop/datasets/Expression_Data/KM.DI1.20.tiff extracted
/home/ankitsingh/Desktop/datasets/Expression_Data/TM.NE3.179.tiff extracted
/home/ankitsingh/Desktop/datasets/Expression_Data/NM.SU3.103.tiff extracted
/home/ankitsingh/Desktop/datasets/Expression_Data/MK.SU1.122.tiff extracted
/home/ankitsingh/Desktop/datasets/Expression_Data/MK.FE3.133.tiff extracted
/home/ankitsingh/Desktop/datasets/Expression_Data/KA.DI2.43.tiff extracted
/home/ankitsingh/Desktop/datasets/Expression_Data/KR.FE1.89.tiff extracted
/home/ankitsingh/Desktop/datasets/Expression_Data/KA.AN2.40.tiff extracted
/home/ankitsingh/Desktop/datasets/Expression_Data/KM.NE2.2.tiff extracted
/home/ankitsingh/De

In [27]:
#For generating face embeddings using a pretrained model based on facenet

from numpy import load
from numpy import expand_dims
from numpy import asarray
from numpy import savez_compressed
from keras.models import load_model

def get_embedding(model, face_pixels):
    face_pixels = face_pixels.astype('float32')
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean) / std
    samples = expand_dims(face_pixels, axis=0)
    yhat = model.predict(samples)
    return yhat[0]

data = load('expression-dataset.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Loaded: ', trainX.shape, trainy.shape, testX.shape, testy.shape)
model = load_model('facenet_keras.h5')
print('Loaded Model')
newTrainX = list()
for face_pixels in trainX:
    embedding = get_embedding(model, face_pixels)
    newTrainX.append(embedding)
newTrainX = asarray(newTrainX)
print(newTrainX.shape)
newTestX = list()
for face_pixels in testX:
    embedding = get_embedding(model, face_pixels)
    newTestX.append(embedding)
newTestX = asarray(newTestX)
print(newTestX.shape)
savez_compressed('expression-embeddings.npz', newTrainX, trainy, newTestX, testy)

Loaded:  (138, 160, 160, 3) (138,) (75, 160, 160, 3) (75,)


/home/ankitsingh/.local/lib/python3.6/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Loaded Model
(138, 128)
(75, 128)


In [29]:
#Model Training

from numpy import load
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

data = load('expression-embeddings.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Dataset: train=%d, test=%d' % (trainX.shape[0], testX.shape[0]))

in_encoder = Normalizer(norm='l2')
trainX = in_encoder.transform(trainX)
testX = in_encoder.transform(testX)

out_encoder = LabelEncoder()
out_encoder.fit(trainy)
trainy = out_encoder.transform(trainy)
testy = out_encoder.transform(testy)

tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]},
                    {'kernel': ['poly'],
                     'C': [1, 10, 100, 1000]}
                   ]

#Using GridSearchCV to find the best tuned model
clf = GridSearchCV(SVC(), tuned_parameters, cv=2, scoring='accuracy')
clf.fit(trainX, trainy)

yhat_train = clf.predict(trainX)
yhat_test = clf.predict(testX)

Dataset: train=138, test=75


In [33]:
#Model evaluation

from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
print('Training F1 score: ', f1_score(trainy, yhat_train, average='micro'))
print('Testing F1 score: ', f1_score(testy, yhat_test, average='micro'))

print('Classification Report: ', classification_report(testy, yhat_test))

Training F1 score:  1.0
Testing F1 score:  0.8133333333333334
Classification Report:                precision    recall  f1-score   support

           0       0.90      0.82      0.86        11
           1       0.92      1.00      0.96        11
           2       0.50      0.75      0.60         8
           3       0.89      1.00      0.94         8
           4       0.77      1.00      0.87        10
           5       0.75      0.46      0.57        13
           6       1.00      0.79      0.88        14

    accuracy                           0.81        75
   macro avg       0.82      0.83      0.81        75
weighted avg       0.83      0.81      0.81        75

